In [1]:
import requests
import pandas as pd 
import numpy as np
from datetime import datetime
import os

In [15]:

def connect_to_api(api):
    response  = requests.get(api)
    result = response.status_code

    if result == 200:
        print('Connect to API Success')
        data = response.json()
    elif result == 404:
        print('Fail to connect API')
    return result , data



In [17]:
api_transfer = 'https://www.fotmob.com/api/transfers'
api_league = 'https://www.fotmob.com/api/allLeagues'
data = connect_to_api(api_transfer)[1]
data_league = connect_to_api(api_league)[1]

Connect to API Success
Connect to API Success


In [4]:
data.keys()

dict_keys(['transfers', 'hits'])

In [5]:
def processing_data(data):
    raw_data = []
    # xử lý json format to dict -> DF
    for i in range(len(data['transfers'])):
        a =     {
            'playerId' : data['transfers'][i]['playerId'],
            
            'name' : data['transfers'][i]['name'],
            'position': data.get('transfers')[i].get('position'),
            'transferDate': data['transfers'][i]['transferDate'],
            'fromClubId': data['transfers'][i]['fromClubId'],
            'fromClub': data['transfers'][i]['fromClub'],
            'toClub': data['transfers'][i]['toClub'],
            'toClubId': data['transfers'][i]['toClubId'],
            'fromDate': data['transfers'][i]['fromDate'], 
            'toDate': data.get('transfers')[i].get('toDate'),
            'transferType': data.get('transfers')[i].get('transferType'),
            'marketValue': data.get('transfers')[i].get('marketValue'),
            'fee': data.get('transfers')[i].get('fee')
            

            

        }
        raw_data.append(a)
        # break
    df = pd.DataFrame(raw_data)

    df['position_name'] = df['position'].str['label']
    df['position_description'] = df['position'].str['key'].str.split('_').str[0]
    df['transferType'] = df['transferType'].str['text']
    df['fee_transfer'] = df['fee'].str['value'].replace(np.nan,0)
    df['marketValue'] = df['marketValue'].replace(np.nan,0)
    df['transferDate'] =  pd.to_datetime(df['transferDate']).dt.strftime('%d-%m-%Y')
    df['fromDate'] =  pd.to_datetime(df['fromDate']).dt.strftime('%d-%m-%Y')
    df['toDate'] =  pd.to_datetime(df['toDate']).dt.strftime('%d-%m-%Y')


    # df.head(5)
    return df




In [6]:
transfer_table = processing_data(data)
transfer_table.head()

,playerId,name,position,transferDate,fromClubId,fromClub,toClub,toClubId,fromDate,toDate,transferType,marketValue,fee,position_name,position_description,fee_transfer
0,1649302,Jayden Hibbert,None,19-06-2024,773958,Atlanta United,Birmingham Legion FC,-1,17-06-2024,31-12-2024,on loan,25000.0,"{'feeText': 'on loan', 'localizedFeeText': 'on...",None,None,0.0
1,877450,Lasse Sørensen,"{'label': 'RM', 'key': 'rightmidfielder_short'}",19-06-2024,8430,Lincoln,Huddersfield,9796,01-07-2024,30-06-2027,contract,550000.0,"{'feeText': 'fee', 'localizedFeeText': 'transf...",RM,rightmidfielder,470000.0
2,607331,Alistair Coote,"{'label': 'LW', 'key': 'leftwinger_short'}",19-06-2024,2,Free agent,Shelbourne,5751,01-07-2024,NaN,contract,130000.0,"{'feeText': 'free transfer', 'localizedFeeText...",LW,leftwinger,0.0
3,568581,Alex Bass,"{'label': 'GK', 'key': 'keeper'}",19-06-2024,8472,Sunderland,Notts County,9819,01-07-2024,30-06-2027,contract,240000.0,None,GK,keeper,0.0
4,832826,Sam Bone,"{'label': 'CB', 'key': 'centerback_short'}",19-06-2024,8131,Maidstone United,Shelbourne,5751,01-07-2024,30-11-2024,contract,110000.0,"{'feeText': 'free transfer', 'localizedFeeText...",CB,centerback,0.0


In [7]:
date_transfer = list(transfer_table['transferDate'].unique())
date_transfer

['19-06-2024',
 '18-06-2024',
 '17-06-2024',
 '16-06-2024',
 '15-06-2024',
 '14-06-2024']

In [14]:
for i in date_transfer:
    file = transfer_table[transfer_table['transferDate'] == i]
    date = datetime.strptime(i,"%d-%m-%Y")
    year = date.year
    month = date.month
    day = date.day
    
    path = f'data/transfer/{year}/{month}'
    file_path = f'{path}/{i}.csv'
    # Create directory if not exists 
    os.makedirs(path,exist_ok=True)
    
    # Write to directory
    file.to_csv(file_path, index=False)
    print(f'Write SUCCESS to: {file_path}')
    # break

Write SUCCESS to: data/transfer/2024/6/19-06-2024.csv
Write SUCCESS to: data/transfer/2024/6/18-06-2024.csv
Write SUCCESS to: data/transfer/2024/6/17-06-2024.csv
Write SUCCESS to: data/transfer/2024/6/16-06-2024.csv
Write SUCCESS to: data/transfer/2024/6/15-06-2024.csv
Write SUCCESS to: data/transfer/2024/6/14-06-2024.csv


,playerId,name,position,transferDate,fromClubId,fromClub,toClub,toClubId,fromDate,toDate,transferType,marketValue,fee,position_name,position_description,fee_transfer
0,1649302,Jayden Hibbert,None,19-06-2024,773958,Atlanta United,Birmingham Legion FC,-1,17-06-2024,31-12-2024,on loan,25000.0,"{'feeText': 'on loan', 'localizedFeeText': 'on...",None,None,0.0


In [41]:
data_league['countries'][0]

{'ccode': 'ALB',
 'name': 'Albania',
 'leagues': [{'id': 10175,
   'name': 'Super Cup',
   'localizedName': 'Super Cup',
   'pageUrl': '/leagues/10175/overview/super-cup'},
  {'id': 260,
   'name': 'Superiore',
   'localizedName': 'Superiore',
   'pageUrl': '/leagues/260/overview/superiore'},
  {'id': 9173,
   'name': 'Superiore Qualification',
   'localizedName': 'Superiore Qualification',
   'pageUrl': '/leagues/9173/overview/superiore-qualification'}],
 'localizedName': 'Albania'}

In [52]:
league = data_league['countries']
df1 = pd.DataFrame(league)
df1.head()

,ccode,name,leagues,localizedName
0,ALB,Albania,"[{'id': 10175, 'name': 'Super Cup', 'localized...",Albania
1,ALG,Algeria,"[{'id': 516, 'name': 'Ligue 1', 'localizedName...",Algeria
2,ARG,Argentina,"[{'id': 112, 'name': 'Liga Profesional', 'loca...",Argentina
3,ARM,Armenia,"[{'id': 118, 'name': 'Premier League', 'locali...",Armenia
4,AUS,Australia,"[{'id': 113, 'name': 'A-League', 'localizedNam...",Australia


In [67]:
df1['leagues']

0     [{'id': 10175, 'name': 'Super Cup', 'localized...
1     [{'id': 516, 'name': 'Ligue 1', 'localizedName...
2     [{'id': 112, 'name': 'Liga Profesional', 'loca...
3     [{'id': 118, 'name': 'Premier League', 'locali...
4     [{'id': 113, 'name': 'A-League', 'localizedNam...
                            ...                        
86    [{'id': 130, 'name': 'MLS', 'localizedName': '...
87    [{'id': 10342, 'name': 'Cup', 'localizedName':...
88    [{'id': 339, 'name': 'Primera Division', 'loca...
89    [{'id': 9088, 'name': 'V-League', 'localizedNa...
90    [{'id': 116, 'name': 'Premier League', 'locali...
Name: leagues, Length: 91, dtype: object

In [84]:
def get_league(league):
    concat_name = []
    for l in league:
        name = l['name']
        concat_name.append(name)
    return concat_name


In [85]:
df2['leagues'].apply(get_league)

26    [Premier League, Championship, League One, Lea...
Name: leagues, dtype: object

In [86]:
df2 = df1[df1['name'] == 'England']
df2


,ccode,name,leagues,localizedName
26,ENG,England,"[{'id': 47, 'name': 'Premier League', 'localiz...",England


In [89]:
for i in df2['leagues']:
    for a in i:
        print(a)

{'id': 47, 'name': 'Premier League', 'localizedName': 'Premier League', 'pageUrl': '/leagues/47/overview/premier-league'}
{'id': 48, 'name': 'Championship', 'localizedName': 'Championship', 'pageUrl': '/leagues/48/overview/championship'}
{'id': 108, 'name': 'League One', 'localizedName': 'League One', 'pageUrl': '/leagues/108/overview/league-one'}
{'id': 109, 'name': 'League Two', 'localizedName': 'League Two', 'pageUrl': '/leagues/109/overview/league-two'}
{'id': 117, 'name': 'National League', 'localizedName': 'National League', 'pageUrl': '/leagues/117/overview/national-league'}
{'id': 8944, 'name': 'National North & South', 'localizedName': 'National North & South', 'pageUrl': '/leagues/8944/overview/national-north-south'}
{'id': 8947, 'name': 'Premier Division', 'localizedName': 'Premier Division', 'pageUrl': '/leagues/8947/overview/premier-division'}
{'id': 9084, 'name': 'Premier League 2 Div 1', 'localizedName': 'Premier League 2 Div 1', 'pageUrl': '/leagues/9084/overview/premie